In [3]:
import numpy as np

#Nagel & Schreckenberg traffic simulation addapted from class that I'll use for each lane

class Lane:

    def __init__(self, road_length=100, car_density=0.2, prob_slow=0.5,
                 max_speed=5):
        '''
        Create a new traffic simulation object. Cars are distributed randomly
        along the road and start with random velocities.

        Inputs:

            road_length (int) The number of cells in the road. Default: 100.

            car_density (float) The fraction of cells that have a car on them.
              Default: 0.2.

            prob_slow (float) The probability that a car will randomly slow down
              by 1 during an update step. Default: 0.5.

            max_speed (int) The maximum speed in car cells per update step.
              Default: 5.
        '''
        self.road_length = road_length
        self.car_density = car_density
        self.max_speed = max_speed
        self.prob_slow = prob_slow

        # Create an empty road: -1 means empty in this simulation
        self.state = np.full(self.road_length, -1, dtype=int)
        # Choose random locations to place cars
        random_indexes = np.random.choice(
            range(self.road_length),
            size=int(round(car_density * self.road_length)),
            replace=False)
        # Give each car a random initial speed from 0 to max_speed
        self.state[random_indexes] = np.random.randint(
            0, self.max_speed + 1, size=len(random_indexes))

        # Keep track of the time steps and average traffic flow at each step
        self.time_step = 0
        self.average_traffic_flow = []

    def update(self, optimized=True):
        '''
        Advance one time step in the simulation.
        '''
        if optimized:
            return self.update_optimized()
        
        # Update car speeds
        for i in range(self.road_length):
            if self.state[i] != -1:
                # Distance is the number of empty cells between this car and the
                # next car plus 1
                distance = 1
                while self.state[(i + distance) % self.road_length] == -1:
                    distance += 1
                # Acceleration
                if self.state[i] < self.max_speed:
                    self.state[i] += 1
                # Deceleration
                if self.state[i] >= distance:
                    self.state[i] = distance - 1
                # Randomization
                if (
                    (self.state[i] > 0) and
                    (np.random.uniform() < self.prob_slow)
                ):
                    self.state[i] -= 1

        # Move cars forward using their new speeds
        new_state = np.full(self.road_length, -1, dtype=int)
        for i in range(self.road_length):
            cell = self.state[i]
            if cell != -1:
                new_state[(i + cell) % self.road_length] = cell
        self.state = new_state

        # Update average traffic flow history
        self.average_traffic_flow.append(
            sum(self.state[self.state > 0]) / self.road_length)
        self.time_step += 1

    def update_optimized(self):
        '''
        Advance one time step in the simulation.
        '''
        # Get the location of each car
        car_indexes = np.where(self.state >= 0)[0]
        
        # If there are no cars, we have nothing to update
        if len(car_indexes) == 0:
            self.average_traffic_flow.append(0)
            self.time_step += 1
            return

        # Get the speed of each cars
        car_speeds = self.state[car_indexes]
        # Get the distance from each car to the next car
        car_distances = np.ediff1d(
            np.concatenate((car_indexes,
                            [car_indexes[0] + self.road_length])))
        # Update car speeds
        # Acceleration
        car_speeds = np.where(
            car_speeds + 1 <= self.max_speed,
            car_speeds + 1,
            self.max_speed)
        # Deceleration
        car_speeds = np.where(
            car_speeds >= car_distances,
            car_distances - 1,
            car_speeds)
        # Randomization
        moving_car_indexes = np.where(car_speeds > 0)[0]
        car_speeds[moving_car_indexes] -= (
            np.random.uniform(size=len(moving_car_indexes)) < self.prob_slow)

        # Move cars forward using their new speeds
        car_indexes += car_speeds
        car_indexes %= self.road_length
        self.state.fill(-1)
        self.state[car_indexes] = car_speeds

        # Update traffic flow history
        self.average_traffic_flow.append(sum(car_speeds) / self.road_length)
        self.time_step += 1

    def display(self):
        '''
        Print out the current state of the simulation.
        '''
        print(''.join('.' if x == -1 else str(x) for x in self.state))


sim = Lane(
    road_length=50, car_density=0.2, max_speed=5, prob_slow=0.5)
sim.state = sim.state.copy()
for i in range(10):
    sim.display()
    sim.update(optimized=True)

.2....1....02.......4.1................4..2...4..2
1...3...2..0...3....0..1.................2..2...2.
..2....3.1..1......40...1.................1....30.
.....3..1..2..2....00....1..................2..0.1
.2.....2.1..1....3.0.1....1...................20..
...2...0...2..2...10..1.....2.................0.1.
.....2..1...1....300....2......3..............0..1
1......2..2...2..000.......3......3............1..
..2.....1...2..1.000..........3......3...........2
.2...3...1....20.00.1............3......3.........


In [4]:
#class for connecting multiple lanes 

class Nodes:
    
    def __init__(self, lanes:dict([str,Lane])):
        self.lanes = lanes
        self.queue = {}

    def addToQueue(self, dir):
        self.queue.append(dir)

    def removeFromQueue(self):
        dirFrom = self.queue.pop()

        arr = np.array(self.lanes.keys())
        boolArr = arr != dirFrom
        newArr = arr[boolArr]
        newArr = arr[arr != dirFrom]

        dirTo = np.random.choice(newArr)
        self.lanes[dirTo].

        

In [ ]:
#class for running everything

class System:
    
    def __init__(self, nodes={}):
        self.nodes = nodes